# se instala apache airflow 

In [ ]:
!pip install apache-airflow
!pip install apache-airflow-upgrade-check

In [ ]:
!pip install 'apache-airflow[postgres,google]==2.5.0' \--constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.5.0/constraints-3.7.txt"

# se importan las bibliotecas principales principales

# funcion etl que integra los tres procesos de transformacion

In [50]:

!pip install tweepy
!pip install --upgrade tweepy
!pip show tweepy
import tweepy
from tweepy import API
import re
from tweepy import Stream
import pandas as pd
from time import sleep
from random import randint
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.metrics import *
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

def etl():
  """Collect  tweets matching the given query related with energy markets. with min likes and min retweets
  

   
    query (str): The search query to use with specific words  like invest or up down electricity 
    max_tweets (int): The maximum number of tweets to collect.
    min_retweets just to filter tweets with good amount of retwwets  which is set to 1000
  
  """
  import config
  consumer_key = config.consumer_key
  consumer_secret = config.consumer_secret
  access_token = config.access_token
  access_token_secret = config.access_token_secret
  

  words=["positive","Invest", "up","down","Price up","win","stock","fired"]

  query=["Invest"or"up"or"Energy"or"Price up"or"win"or"stock"or"Natural Gas"or"coal"]



  
  min_likes=0
  min_retweets=1000
  auth = tweepy.OAuth1UserHandler(
      consumer_key, consumer_secret, access_token, access_token_secret
  )

  api = tweepy.API(auth)



  
  api = tweepy.API(auth, wait_on_rate_limit=True)



 

# search for tweets since the time 15 minutes ago


  

#extract and log 
  logging.info("extrayendo ")
  





  
  
  #create df
  df = pd.DataFrame(columns=['created_at', 'text', 'screen_name', 'retweets', 'favorites','negative sentiment', 'positive sentiment', "#words"])
  # create a sentiment analyzer
  sid = SentimentIntensityAnalyzer()
  
  # create a dictionary to store the results
  
  processed_tweets = []
  contador_Palabras=0
  logging.info("transformando")
  tweets = tweepy.Cursor(api.search_tweets,tweet_mode='extended', q=query, lang='en')
  for tweet in tweets:
    #tweets.append(tweet.text)
    if tweet.favorite_count >= min_likes and tweet.retweet_count >= min_retweets:
      #print(tweet.text)
      # en esta etapa se tokeniza el tweet para y poder limpiarlo por tipo de palabra
      words = nltk.word_tokenize(re.sub(r'https?://\S+', '', tweet.text))
      #words = nltk.word_tokenize( tweet.text)
      #  remove the punctuation
      words = [word for word in words if word.isalpha()]
      
      # words to lowercase
      words = [word.lower() for word in words]

      #  remove the stop words from the tweet
      stop_words = nltk.corpus.stopwords.words('english')
      words = [word for word in words if word not in stop_words]
      contador_Palabras=len(words)
      # Stemming en esta etapa se llevan las palabras a su raiz 
      stemmer = nltk.SnowballStemmer('english')
      words = [stemmer.stem(word) for word in words]
      processed_tweets.append(words)
      #print(type())
      sentiment1 = sid.polarity_scores(tweet.text)
      scores = []

      scores.append(sid.polarity_scores(tweet.text)['compound'])

            
            
      df = df.append({
                      'created_at': tweet.created_at,
                      'text': tweet.text,
                      'screen_name': tweet.user.screen_name, 
                      'retweets': tweet.retweet_count,
                      'favorites': tweet.favorite_count,
                      'negative sentiment':min(scores),
                      'positive sentiment':max(scores),
                      "#words":contador_Palabras,

                      }, ignore_index=True)
      
      
          
  # en esta etapa se carga el df como csv para su posterior visulaizacion en power BI
  logging.info("cargando")
  df.to_csv("data1.csv", index=False)
  


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: tweepy
Version: 4.12.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: oauthlib, requests, requests-oauthlib
Required-by: 


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#DAG en esta celda se delclaran los atributos del dag y el task que debe realizar 


In [51]:
import logging
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
import psycopg2
from datetime import datetime, timedelta

default_args={
    'owner': 'enersinc, carlos saenz',
    'email':['carlosaenz.26@hotmail.com'],
    'depends_on_past':False,
    'email_on_failure':['carlosaenz.26@hotmail.com'],
    'retries':3,
    'retry_delay': timedelta(minutes=5),
} 

dag=DAG(
    'twitter_ETL',
    default_args=default_args,
    description='tweets investmen ETL enersinc prueba Carlos S',
    schedule_interval=timedelta(days=3),
    start_date=days_ago(2),

)

twiter_etl=PythonOperator(task_id="complete_etl",
                              python_callable=etl
                            ,dag=dag)



twiter_etl


<ipython-input-51-7c7f9ced12db>:22 RemovedInAirflow3Warning: Function `days_ago` is deprecated and will be removed in Airflow 3.0. You can achieve equivalent behavior with `pendulum.today('UTC').add(days=-N, ...)`

<ipython-input-51-7c7f9ced12db>:17 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): complete_etl>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

          ┌──────────┐
          │ Autenticación │
          └──────────┘
                 ▲
                 └──────┐
          ┌──────────┐  │
          │ Extracción de Tweets │
          └──────────┘  │
                 ▲      └──────┐
                 └──────┐  ┌─────────────┐
      ┌──────────────┐  │  │Extracción de│ 
      │              │     │tweets por palabras
                               claves    │ 
      │              │     │             │
      Transformacion de │  └─────────────┘
      │ tweets   │
      └─────────────┘  │  
                 ▲      └──────┐
                 └──────┐  ┌──────────────────┐
          ┌──────────┐  │  │ Almacenamiento   │
          └──────────┘     └──────────────────┘
